### Being a data scientist, the real sh*t:

* Data scientists in real life have multiple goals that fit under the
  general category of "making sense of data" or "turning data into
  insights".
  * Often we are focused on building a predictive model, and on
    maximizing the predictive model's accuracy. Sometimes this focus
    is overemphasized. This is a thing that we do, but this is not the
    only thing.
    * Things that fail are often still interesting insights.
    * Anecdotes are often interesting insights.
    * The utility of an analysis is independent of the sophistication
      of the algorithm. Sometimes the most mind-blowing insights come
      from lists, tables, histograms, or scatter plots. Don't throw
      out cool stuff that isn't technically advanced unless absolutely
      necessary.
      * Don't over-design and under-deliver. For every data science
        project that you see or hear about, the version in the data
        scientist's head was probably fancier, bigger, more
        comprehensive, more elegant, presented in a cooler format, or
        with better copy, et cetera, ad nauseum. The reason you heard
        about it at all, however, is because it was *finished*, and
        published or released in all its heart-wrenching
        imperfection.
        * Start with something small, and build from there, as
        necessary, as time allows.
        * Jot down the elaborations, next steps, uh-ohs, or grand
          ideas that strike you as you are working. Leave them alone
          for a while and then come back and look at them later.
          * Many things that feel like huge "uhoh"s in the heat of the
            moment are actually small deals or even false alarms. The
            fewer of these you spend time on, the better.




In [160]:
import requests
import re
import time
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

In [208]:
my_html = """
<html>
<head>
</head>
<body>
<Div style="border: 1px solid">
There isn't much in this file, except a list of to-do items

<ul>
  <li>Feed the cat</li>
  <li>Wash the dished</li>
  <li>Make coffee</li>
  <li>Go to the store</li>
  <li>Write BeautifulSoup lecture</li>
</ul>
</div>
</body>
</html>
"""

In [209]:
display(HTML(my_html))

In [210]:
#school-begining page:  http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=0&max=20

#professor: http://www.ratemyprofessors.com/ShowRatings.jsp?tid=7503

# school_url_flex used to generate link of different pages

school_url="http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=0&max=20"
school_url_flex="http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset={}&max=20"
prof_url="http://www.ratemyprofessors.com/ShowRatings.jsp?tid=7503"
prof_url_flex="http://www.ratemyprofessors.com{}"

prof_response=requests.get(prof_url)
school_response=requests.get(school_url)

print(prof_response.status_code)
print(school_response.status_code)



200
200


# Berkerly page 1: http://www.ratemyprofessors.com/search.jsp?queryoption=HEADER&queryBy=teacherName&schoolName=University+of+California+Berkeley&schoolID=1072&query=*
#berkerly page2: http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=20&max=20
berkerly page3: http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=40&max=20
# not useful when making max=1000
http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=20&max=20

# Binghamton: http://www.ratemyprofessors.com/search.jsp?queryoption=HEADER&queryBy=teacherName&schoolName=Binghamton+University+%28SUNY%29&schoolID=958&query=*


In [211]:
display(HTML(prof_response.text))

In [212]:
display(HTML(school_response.text))

In [214]:
prof_page=prof_response.text
school_page=school_response.text

prof_soup = BeautifulSoup(prof_page,"lxml")
school_soup = BeautifulSoup(school_page,"lxml")

In [215]:
# total number of professor listings in a school. Given school soup, retun number of professors

def total_professors(school_soup):
    
    for e in school_soup.find_all(class_="toppager"):     #(class_="toppager-left"):
        temp=e.find(class_="result-count").text
       # name=e.find(class_="pfname").text.strip()+" "+e.find(class_="plname").text.strip()
        result=re.findall(r'\d+', temp)
        return int(max(result)) #usually the pages shows 1-20 records out of x result. x would be the maximum of the three number

total_prof=total_professors(school_soup)
total_prof

3986

In [216]:
def page_of_listing(total_professors):
    pages=total_professors//20+1
    return pages

pages=page_of_listing(total_prof)
pages

200

In [217]:
# given school's flexible url and pages of listing, generae a list of urls of all webpage pages
def page_urls(url_flex, pages):
    list_urls=[]
    for i in range (1,pages+1):
        offset=i*20
        page_url=url_flex.format(offset)
        list_urls.append(page_url)
    return list_urls

urls=page_urls(school_url_flex, pages)
urls




['http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=20&max=20',
 'http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=40&max=20',
 'http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=60&max=20',
 'http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=80&max=20',
 'http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=100&max=20',
 'http://www.ratemy

In [218]:
# generate links for professsors. Give page_links, find out a list of links for professors


link2=page_urls(school_url_flex, 1)
def prof_urls(page_links, url_flex):
    url_listing=[]
    for link in page_links:
        temp_response=requests.get(link)
        temp_page=prof_response.text
        temp_soup = BeautifulSoup(temp_page,"lxml")
        for e in school_soup.find_all('li',class_="listing PROFESSOR"):  
            temp= e.find('a')['href']
            prof_url=prof_url_flex.format(temp)
            url_listing.append(prof_url)
    return url_listing
                    
    
prof_urls(link2, prof_url_flex)

['http://www.ratemyprofessors.com/ShowRatings.jsp?tid=628',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=814',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=7503',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=7745',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=7746',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=8307',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=9281',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=11539',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=12105',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=12108',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=12109',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=12110',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=12701',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=13904',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=13905',
 'http://www.ratemyprofessors.com/ShowRatings.jsp?tid=16984',
 'http://www.rate

In [219]:
# grab professor name. Given school soup, retun string

def get_name(prof_soup):
    for e in prof_soup.find_all(class_="profname"):
        name=e.find(class_="pfname").text.strip()+" "+e.find(class_="plname").text.strip()
    return name

get_name (prof_soup)

'Bob Calonico'

In [220]:
# Grab the rating of professor. Given Soup, retun float

def get_rating(prof_soup):
    
    temp=prof_soup.find(class_="breakdown-container quality")
    s=temp.find(class_='grade')
    return float(s.text)
            
            
get_rating(prof_soup)
        
# can be list of strings

4.7

In [221]:
# Grab the level of difficulty of the professor. iven Soup, retun float

def get_level_of_difficulty(prof_soup):
    
    level_of_difficulty=prof_soup.find(class_="breakdown-section difficulty").stripped_strings

    return float(list (level_of_difficulty)[1])

get_level_of_difficulty(prof_soup)
    


1.6

In [222]:

'''
test=[]
for element in soup.find_all(class_="tag-box"):  #entire tag
    for e in element.find_all(class_='tag-box-choosetags'): #each tab was embed in tag-box-choosetags class
        test.append (list(e.stripped_strings))
test

'''

'\ntest=[]\nfor element in soup.find_all(class_="tag-box"):  #entire tag\n    for e in element.find_all(class_=\'tag-box-choosetags\'): #each tab was embed in tag-box-choosetags class\n        test.append (list(e.stripped_strings))\ntest\n\n'

In [304]:
# get all tags of professor
tags_url="http://www.ratemyprofessors.com/AddRating.jsp?tid=9670"
tags_page=requests.get(tags_url).text
tags_soup = BeautifulSoup(tags_page,"lxml")

all_tags=[]
for e in tags_soup.find_all('div', class_="scrollable"):  #entire tag
    for f in e.find_all('a',class_=''): #each tag was embed in tag-box-choosetags class
        all_tags.append (f.text.strip().capitalize())
all_tags.sort()
all_tags


['Accessible outside class',
 'Amazing lectures',
 'Beware of pop quizzes',
 'Caring',
 'Clear grading criteria',
 'Extra credit',
 'Get ready to read',
 'Gives good feedback',
 'Graded by few things',
 'Group projects',
 'Hilarious',
 'Inspirational',
 'Lecture heavy',
 'Lots of homework',
 'Participation matters',
 'Respected',
 "Skip class? you won't pass.",
 'So many papers',
 'Test heavy',
 'Tough grader']

In [303]:
'''
all_tags=['Tough Grader',
 'Gives good feedback',
 'Respected',
 'Get ready to read',
 'Participation matters',
 "Skip class? You won't pass.",
 'LOTS OF HOMEWORK',
 'Inspirational',
 'BEWARE OF POP QUIZZES',
 'ACCESSIBLE OUTSIDE CLASS',
 'SO MANY PAPERS',
 'Clear grading criteria',
 'Hilarious',
 'TEST HEAVY',
 'GRADED BY FEW THINGS',
 'Amazing lectures',
 'Caring',
 'EXTRA CREDIT',
 'GROUP PROJECTS',
 'LECTURE HEAVY']
'''
# Grab the main tags of the professor. Given soup, return tags


def get_tags(prof_soup):
    list_of_tags=[]
    dic={}
    total_count=0
    for e in prof_soup.find_all(class_="tag-box"):  #entire tag
        for f in e.find_all(class_='tag-box-choosetags'): #each tag was embed in tag-box-choosetags class
            list_of_tags.append (f.text.strip())
    
    # sort result list for efficiency
    list_of_tags.sort()
    
    #split text and add the count to each tag
    for i in list_of_tags:
        category=re.findall(r'[^\(]*', i, re.I)[0].strip().capitalize()
        count=int(re.findall(r'\d', i, re.I)[0])
        dic[category]=count
        total_count+=count
    
    #normalize the count for each tag
    for key in dic:
        dic[key]=round(dic[key]/total_count,2)
           
    return dic
    
    
    
have=get_tags(prof_soup)
have



{'Accessible outside class': 0.05,
 'Caring': 0.26,
 'Hilarious': 0.26,
 'Inspirational': 0.32,
 'Respected': 0.11}

In [305]:
# All tags is a lit and repreents entire listing of tags. Have_tags is a dictionary with normalized count of tags

def tag_count(all_tags, have_tags):
    dic={}
    for key in all_tags:
        dic[key]=0
    
    for key in have_tags:
        if key in all_tags:
            dic[key]=have_tags[key]
    
    return dic

tag_count(all_tags, have)
        
    

{'Accessible outside class': 0.05,
 'Amazing lectures': 0,
 'Beware of pop quizzes': 0,
 'Caring': 0.26,
 'Clear grading criteria': 0,
 'Extra credit': 0,
 'Get ready to read': 0,
 'Gives good feedback': 0,
 'Graded by few things': 0,
 'Group projects': 0,
 'Hilarious': 0.26,
 'Inspirational': 0.32,
 'Lecture heavy': 0,
 'Lots of homework': 0,
 'Participation matters': 0,
 'Respected': 0.11,
 "Skip class? you won't pass.": 0,
 'So many papers': 0,
 'Test heavy': 0,
 'Tough grader': 0}

In [225]:
testing_urls=page_urls(school_url_flex, 3)
testing_urls

['http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=20&max=20',
 'http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=40&max=20',
 'http://www.ratemyprofessors.com/search.jsp?query=&queryoption=HEADER&stateselect=&country=&dept=&queryBy=teacherName&facetSearch=&schoolName=University+of+California+Berkeley&offset=60&max=20']

In [239]:
a=[1,2,3]
a.append(1)
a.append([2,3,6])
print (a)
a.extend([1,4,5])
print (a)

[1, 2, 3, 1, [2, 3, 6]]
[1, 2, 3, 1, [2, 3, 6], 1, 4, 5]


In [229]:

total_prof=total_professors(school_soup)
pages=page_of_listing(total_prof)

testing_urls=page_urls(school_url_flex, 1)

print (total_prof, pages)

def scrape_data(urls):
    data=[]
    temp=[]
    for link in urls:
        temp_page_links=prof_urls(urls, prof_url_flex)
        for link2 in temp_page_links:
            temp_response=requests.get(link2)
            temp_page=temp_response.text
            temp_soup = BeautifulSoup(temp_page,"lxml")
            temp_name=get_name (temp_soup)
            temp_rating=get_rating(temp_soup)
            temp_difficulty=get_level_of_difficulty(temp_soup)
            temp_tag=get_tags(temp_soup)            
            
            temp.extend(temp_name, temp_rating, temp_difficulty, temp_tag)
            data.extend(temp_name, temp_rating, temp_difficulty, temp_tag])
            time.sleep(11)
        return data

result=scrape_data(testing_urls)


        
        
    
    

3986 200


TypeError: append() takes exactly one argument (3 given)

In [207]:
result

[['Jerry Sanders', 2.7, 3.4, []],
 ['Chris Dolder',
  4.8,
  3.9,
  [' Gives good feedback (1)', ' Inspirational (1)']],
 ['Bob Calonico',
  4.7,
  1.6,
  [' Inspirational (6)',
   ' Caring (5)',
   ' Hilarious (5)',
   ' Respected (2)',
   ' ACCESSIBLE OUTSIDE CLASS (1)']],
 ['Zack Powell', 2.1, 3.0, []],
 ['Mary Kelsey',
  4.6,
  1.6,
  [' EXTRA CREDIT (7)',
   ' Caring (3)',
   ' Respected (3)',
   ' Participation matters (2)',
   ' Inspirational (2)',
   ' Clear grading criteria (2)',
   ' Tough Grader (1)',
   ' LECTURE HEAVY (1)',
   ' Gives good feedback (1)',
   ' Amazing lectures (1)',
   " Skip class? You won't pass. (1)",
   ' Get ready to read (1)',
   ' ACCESSIBLE OUTSIDE CLASS (1)']],
 ['John Searle',
  3.6,
  3.1,
  [' Hilarious (3)',
   ' Amazing lectures (3)',
   ' Respected (3)',
   ' Get ready to read (2)',
   ' Tough Grader (1)',
   ' Caring (1)',
   ' Inspirational (1)']],
 ['Jean Frechet', 4.4, 3.3, []],
 ['Phillip Stark', 2.9, 3.0, []],
 ['William Hanks',
  4.4,


In [ ]:
import pickle

filename = '/Users/xzhou/github/project_files/project_luther/professor_data.pkl'

try:
    with open(filename,'rb') as pklfile:
        df = pickle.load(pklfile)
except:
    with open(filename,'wb') as pklfile:
        df = pickle.dump(df, pklfile)
